In [31]:
import numpy as np
from scipy import sparse
cost='''11 13 17 14
16 18 14 10
21 24 13 10'''

cost=np.array([row.split(" ") for row in cost.split("\n")],dtype=int)
demand=[250,300,400]
supply=[200,225,275,250]
cost
# np.random.seed(1254)
# cost=np.random.randint(low=100,high=200,size=(100,1000))
# cost[cost>150]=0
# sparsity=np.copy(cost)
# sparsity[sparsity>1]=1
# requests=np.random.randint(low=20,high=70,size=(100,1000))
# allocation=sparsity*requests
# demand=allocation.sum(axis=1)
# supply=allocation.sum(axis=0)
#cost=-1*cost
#cost=cost/100000
#cost = sparse.csr_matrix(cost)

array([[11, 13, 17, 14],
       [16, 18, 14, 10],
       [21, 24, 13, 10]])

In [80]:
resize=tuple(map(sum, zip(cost.shape, (1,1))))
flow=np.zeros(resize)
flow[-1,:-1]=supply
flow[:-1,-1]=demand

y=np.ones(resize)
b=np.ones(resize)
flow_bar=np.zeros(resize)

epsilon=1/10

while not np.array_equal(np.sum(flow[:-1,:-1],axis=1),flow[-1,:-1]):
    #bidding phase
    for i in range(cost.shape[0]):
        if flow[i,-1]==np.sum(flow[i,:-1]):continue
        #in sparse case just the relevant columns,rows
        # also not the last row or the unassigned is also substracted here
        pi=cost[i]-y[:,:-1]
        cols=pi.shape[1]
        #remember this trick doesnt work for sparse matrices
        sorted_indices=np.argsort(pi,axis=None)[::-1]
        sorted_pi=list(zip(sorted_indices//cols,sorted_indices%cols))
        kj_m=[]
        sum_f_km_jm=0
        m=0
        while flow[i,-1]-np.sum(flow[i,:-1])>sum_f_km_jm:
            sum_f_km_jm=sum_f_km_jm + flow[sorted_pi[m][0],sorted_pi[m][1]]
            kj_m=kj_m+[sorted_pi[m]]
            m=m+1
        flow_bar[i]=flow[i]
        for bid_poach in kj_m[:-1]:
            flow_bar[i,bid_poach[1]]=flow_bar[i,bid_poach[1]]+flow[i,bid_poach[1]]+flow[bid_poach[0],bid_poach[1]]
        print(kj_m,i)
        flow_bar[i,kj_m[-1][1]]= flow[i,-1]-np.sum(flow_bar[i,:-1])
        w_i=0
        if np.sum(flow_bar[i]>1)>1:
            b[i,:-1]=cost[i,kj_m[-1][1]]-y[kj_m[-1]] + epsilon
        else:
            j1=kj_m[0][1]
            kn_jn=0
            for t in kj_m:
                if t[1] is not j1:
                    kn_jn=t
                    break
            b[i,:-1]=cost[i,kn_jn[-1][1]]-y[kn_jn]+epislon

    print(flow_bar)
    #assignment phase
    for j in range(cost.shape[1]):    
        p_j=np.argsort(b[:-1,j])
        sum_alloc_flow=0
        n=0
        flow[:-1,j]=0
        while sum_alloc_flow < np.sum(flow_bar[:-1,j]):
            sum_alloc_flow=sum_alloc_flow+flow_bar[p_j[n],j]
            flow[p_j[n],j]=flow_bar[p_j[n],j]
            n=n+1
        flow[p_j[n-1],j]= flow[p_j[n-1],j]  - sum_alloc_flow + min(flow[-1,j],np.sum(flow_bar[:-1,j]))
        for i in p_j:
            y[i,j]=b[i,j]
    b=np.ones(resize)
    flow_bar=np.zeros(resize)
    print(flow)

[(3, 2)] 0
[(3, 1), (2, 1), (1, 1), (0, 1), (3, 0)] 1
[(3, 1), (2, 1), (1, 1), (0, 1), (3, 0)] 2
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [175. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [125.   0.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 1), (3, 0)] 2
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [ 50. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0.   0.   0. 250.]
 [  0.   0.   0.   0. 300.]
 [ 50. 225.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 2)] 0
[(3, 1), (1, 1), (0, 1), (3, 0)] 1
[(3, 1)] 2
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [ 50. 125.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [ 50.   0.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 1), (3, 0)] 2
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [125. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0.   0. 

[(3, 2)] 0
[(3, 1), (1, 1), (0, 1), (3, 0)] 1
[(3, 1)] 2
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [125.  50.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [125.   0.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 1), (3, 0)] 2
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [ 50. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0.   0.   0. 250.]
 [  0.   0.   0.   0. 300.]
 [ 50. 225.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 2)] 0
[(3, 1), (1, 1), (0, 1), (3, 0)] 1
[(3, 1)] 2
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [ 50. 125.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [ 50.   0.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 1), (3, 0)] 2
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [125. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0.   0.   0. 250.]
 [  0.   0.   0.   0. 300.]
 

 [200. 225. 275. 250.   0.]]
[(3, 1), (3, 0)] 2
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [125. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0.   0.   0. 250.]
 [  0.   0.   0.   0. 300.]
 [125. 225.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 2)] 0
[(3, 1), (1, 1), (0, 1), (3, 0)] 1
[(3, 1)] 2
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [125.  50.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [125.   0.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 1), (3, 0)] 2
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [ 50. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0.   0.   0. 250.]
 [  0.   0.   0.   0. 300.]
 [ 50. 225.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 2)] 0
[(3, 1), (1, 1), (0, 1), (3, 0)] 1
[(3, 1)] 2
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [ 50. 125.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0. 250.   0. 250.]


[(3, 1)] 2
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [ 50. 125.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [ 50.   0.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 1), (3, 0)] 2
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [125. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0.   0.   0. 250.]
 [  0.   0.   0.   0. 300.]
 [125. 225.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 2)] 0
[(3, 1), (1, 1), (0, 1), (3, 0)] 1
[(3, 1)] 2
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [125.  50.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [125.   0.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 1), (3, 0)] 2
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [ 50. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0.   0.   0. 250.]
 [  0.   0.   0.   0. 300.]
 [ 50. 225.   0.   0. 400.]
 [200. 225. 275. 25

 [200. 225. 275. 250.   0.]]
[(3, 1), (3, 0)] 2
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [ 50. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0.   0.   0. 250.]
 [  0.   0.   0.   0. 300.]
 [ 50. 225.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 2)] 0
[(3, 1), (1, 1), (0, 1), (3, 0)] 1
[(3, 1)] 2
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [ 50. 125.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [ 50.   0.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 1), (3, 0)] 2
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [125. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0.   0.   0. 250.]
 [  0.   0.   0.   0. 300.]
 [125. 225.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 2)] 0
[(3, 1), (1, 1), (0, 1), (3, 0)] 1
[(3, 1)] 2
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [125.  50.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0. 250.   0. 250.]


 [200. 225. 275. 250.   0.]]
[(3, 1), (3, 0)] 2
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [125. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0.   0.   0. 250.]
 [  0.   0.   0.   0. 300.]
 [125. 225.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 2)] 0
[(3, 1), (1, 1), (0, 1), (3, 0)] 1
[(3, 1)] 2
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [125.  50.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [125.   0.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 1), (3, 0)] 2
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [ 50. 225.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0.   0.   0. 250.]
 [  0.   0.   0.   0. 300.]
 [ 50. 225.   0.   0. 400.]
 [200. 225. 275. 250.   0.]]
[(3, 2)] 0
[(3, 1), (1, 1), (0, 1), (3, 0)] 1
[(3, 1)] 2
[[  0.   0. 250.   0. 250.]
 [ 75. 225.   0.   0. 300.]
 [ 50. 125.   0.   0. 400.]
 [  0.   0.   0.   0.   0.]]
[[  0.   0. 250.   0. 250.]


KeyboardInterrupt: 